In [1]:
import os
from pathlib import Path


import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

from pulse import load_all_signals
from pulse import (
    compute_fourier_frequencies,
)
from pulse.wavelet_analysis import plot_dwt_scalogram_plotly

init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

In [2]:
SAMPLING_RATE = 100.0
L = 50
DATA_DIR = Path("../data/signals/")
SIGNAL_FILENAMES = list(DATA_DIR.glob("*"))
N = 1

signals = load_all_signals(DATA_DIR)
print(f"Загружено сигналов: {len(signals)}")
print(f"Размер каждого сигнала: {signals[0].shape if len(signals) > 0 else 'N/A'}")

# df = pd.read_excel("../data/info.xls")

fourier_frequencies = compute_fourier_frequencies(signals, sampling_rate=SAMPLING_RATE)
filtered_signals = np.array(signals)
filtered_signals = np.array(signals)[(40 < fourier_frequencies * 60) & (fourier_frequencies * 60 < 180)]
filtered_names = np.array(SIGNAL_FILENAMES)[(40 < fourier_frequencies * 60) & (fourier_frequencies * 60 < 180)]
filtered_names = [path.name for path in filtered_names]


Загружено сигналов: 827
Размер каждого сигнала: (10000,)


In [3]:
def plot_dwt_scalogram_interactive(idx_signal, wavelet_name):
    clear_output(wait=True)

    signal = filtered_signals[idx_signal]
    signal_name = filtered_names[idx_signal]

    # Получаем данные скалограммы
    scalogram_data = plot_dwt_scalogram_plotly(
        signal=signal,
        wavelet=wavelet_name,
        sampling_rate=SAMPLING_RATE,
        hr_range=(0.8, 1.6),
        title=f"DWT скалограмма: {signal_name}",
    )

    S = scalogram_data["scalogram"]
    freq_ranges = scalogram_data["freq_ranges"]
    hr_levels = scalogram_data["hr_levels"]
    ylabels = scalogram_data["ylabels"]

    # Время для оси X сигнала
    time = np.arange(len(signal)) / SAMPLING_RATE

    # Создаем фигуру с тремя субплотами
    fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=False,
        vertical_spacing=0.1,
        row_heights=[0.25, 0.5, 0.25],
        subplot_titles=(
            f"Пульсовой сигнал: {signal_name}",
            f"DWT скалограмма ({wavelet_name})",
            f"Базисный вейвлет: {wavelet_name}",
        ),
    )

    # 1. Исходный сигнал
    fig.add_trace(
        go.Scatter(x=time, y=signal, mode="lines", name="Сигнал", line=dict(color="blue", width=1)),
        row=1,
        col=1,
    )
    fig.update_yaxes(title_text="Амплитуда", row=1, col=1)

    # 2. Скалограмма
    y_indices = np.arange(len(ylabels))

    fig.add_trace(
        go.Heatmap(
            z=S,
            x=time,
            y=y_indices,
            colorscale="Jet",
            colorbar=dict(title="|Коэффициент|", x=1.02),
            name="Скалограмма",
            hovertemplate="Время: %{x:.2f} с<br>Уровень: D%{y:.0f}<br>Значение: %{z:.2f}<extra></extra>",
        ),
        row=2,
        col=1,
    )

    # Линии диапазона ЧСС
    for hr_level in hr_levels:
        fig.add_shape(
            type="line",
            x0=time[0],
            x1=time[-1],
            y0=hr_level,
            y1=hr_level,
            line=dict(color="white", width=2, dash="dash"),
            row=2,
            col=1,
        )
        fig.add_annotation(
            x=time[-1] * 0.98,
            y=hr_level,
            text="ЧСС",
            showarrow=False,
            xref="x2",
            yref="y2",
            xanchor="left",
            font=dict(color="white", size=10),
            bgcolor="rgba(0,0,0,0.6)",
        )

    fig.update_yaxes(tickmode="array", tickvals=y_indices, ticktext=ylabels, row=2, col=1)
    fig.update_yaxes(title_text="Уровни разложения", row=2, col=1)
    fig.update_xaxes(title_text="Время (с)", row=2, col=1)

    # Настраиваем ось для изображения
    fig.update_xaxes(showticklabels=False, row=3, col=1)
    fig.update_yaxes(showticklabels=False, row=3, col=1)

    # Общие настройки
    fig.update_layout(
        height=1000,
        width=1200,
        title_text=f"Вейвлет-анализ DWT: сигнал {idx_signal}",
        showlegend=False,
    )

    fig.show()


# Виджеты остаются без изменений
wavelet_list_scalogram = ["db4", "db6", "sym5", "sym7", "coif3", "coif5", "haar"]
N_scalogram = len(filtered_signals)

signal_slider_scalogram = widgets.IntSlider(min=0, max=N_scalogram - 1, step=1, value=0, description="Сигнал")

wavelet_dropdown_scalogram = widgets.Dropdown(
    options=wavelet_list_scalogram, value=wavelet_list_scalogram[0], description="Вейвлет"
)


def update_scalogram_plot(idx_signal, wavelet_name):
    plot_dwt_scalogram_interactive(idx_signal, wavelet_name)


out_scalogram = widgets.interactive_output(
    update_scalogram_plot,
    {"idx_signal": signal_slider_scalogram, "wavelet_name": wavelet_dropdown_scalogram},
)

# display(signal_slider_scalogram, wavelet_dropdown_scalogram, out_scalogram)
display(signal_slider_scalogram, wavelet_dropdown_scalogram, out_scalogram)

IntSlider(value=0, description='Сигнал', max=753)

Dropdown(description='Вейвлет', options=('db4', 'db6', 'sym5', 'sym7', 'coif3', 'coif5', 'haar'), value='db4')

Output()